### Plan Tree Creation

#### Steps
- Preprocessing
- Get cheapest access path
- Create plan tree

#### Important Objects
- PlannerInfo - represents a query tree, contain all the information needed to create a plan
- Path - some access path node that represents a single operation on a relation, will be later translated to PlanNode
- RelOptInfo - information about a relation
- PlannedStmt - complete planning information for a single query tree
- PlanNode - final access path picked, building block of a Plan Tree inside a PlannedStmt

### Plan Tree For Single Table Query

#### Preprocessing (simplifying the query)

- Simplifying the query tree - target list (2 + 2 -> 4), clauses (aggregate, sub queries, window, and so on)
- Normalizing boolean expressions - NOT (NOT a) = a
- Flatten boolean expressions - from binary to n-ary 
    ```sql
    (id = 1) OR (id = 2) OR (id = 3)
    ```
    <img src="./helpers/flatten-boolean-expressions.png" alt="drawing" width="900"/>

#### Get Cheapest Access Path

- Create `RelOptInfo` object for the relation which contains: Restrictions (WHERE Clause) + Index List (On relation)
- Add all possible access paths (every access path push by cost based sort to list)
- Get cheapest access path
- Add `LIMIT` / `ORDER BY` costs if necessary

#### Create Plan Tree

Creates `PlannedStmt` object that summarizes the plan and it's meta data, the main attributes are:
- `commandType` - stores a type of operation, such as SELECT, UPDATE or INSERT
- `rtable` - stores rangeTable entries, basically all the relations that the query is dependent on.
- `plantree` - stores a plan tree that is composed of plan nodes, where each node corresponds to a specific operation, such as sequential scan, sort and index scan.

`Plan Tree` - Composed of different `PlanNode`s that all have:
- start-up cost and total_cost - the estimated costs of the operation corresponding to this node.
- rows - the number of rows to be scanned, which is estimated by the planner.
- targetlist - stores the target list items contained in the query tree.
- qual - a list that stores qual conditions.
- lefttree and righttree - the nodes for adding the children nodes.

The root plan node is the last node to be executed.

### Plan Tree For Multi Table Query Additions

#### Preprocessing
- Planning and converting CTEs
- Pulling sub queries up and try to convert to a join
    ```sql
    SELECT * FROM tbl_a AS a, (SELECT * FROM tbl_b) as b WHERE a.id = b.id;
    ```
    ↓
    ```sql
    SELECT * FROM tbl_a AS a, tbl_b as b WHERE a.id = b.id;
    ```

#### Getting The Cheapest Path

##### Standard Way
<img src="./helpers/cheapest-plan-multi-table.png" alt="drawing" width="900"/>

#### Genetic Query Planning
As we saw while reviewing the JOIN operations, there are a lot of options to join two tables and a lot of options for access path for a relation as well.

In a query that should join a lot of tables -> above `geqo_threshold` (default 12) it makes PostgreSQL to waste a lot of resources to find the cheapest plan tree.

That's why genetic query planning has been introduced. It's out of the scope of this course but it using an approximate algorithm to find a reasonable plan in a reasonable time.

#### More Planing Time Optimizations

##### Explicit Join

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres

In [ ]:
# Unexplicit
psql -c "SELECT * FROM a, b, c WHERE a.id = b.id AND b.ref = c.id;"

In [ ]:
# Explicit Inner
psql -c "SELECT * FROM a JOIN (b JOIN c ON (b.ref = c.id)) ON (a.id = b.id);"

In [ ]:
# Explicit Outer
psql -c "SELECT * FROM a LEFT JOIN (b LEFT JOIN c ON (b.ref = c.id)) ON (a.id = b.id);"

`join_collapse_limit` - when does the planner stop to calculate plans for join possibilities?
Set to 1

In [ ]:
TABLES_NUM=20
GEQO_THRESHOLD=21
psql << EOM
    SELECT FORMAT('CREATE TABLE a%s (a INT)', id) FROM generate_series(1,${TABLES_NUM}) AS id
    \gexec
EOM


In [ ]:
# Without limit
psql << EOM
    SET geqo_threshold = ${GEQO_THRESHOLD};
    SET join_collapse_limit = ${GEQO_THRESHOLD};
    WITH cte AS (
    SELECT
        'EXPLAIN SELECT * FROM a1 ' stmt_start, 
        string_agg(FORMAT('JOIN a%s USING (a)', id), ' ') stmt_end
    FROM generate_series(2,${TABLES_NUM}) AS id
    GROUP BY 1
    )
    SELECT CONCAT(stmt_start, stmt_end)
    FROM cte
    \gexec
EOM

In [ ]:
# With limit
JOIN_LIMIT=1
psql << EOM
    SET geqo_threshold = ${GEQO_THRESHOLD};
    SET join_collapse_limit = ${JOIN_LIMIT};
    WITH cte AS (
    SELECT
        'EXPLAIN SELECT * FROM a1 ' stmt_start, 
        string_agg(FORMAT('JOIN a%s USING (a)', id), ' ') stmt_end
    FROM generate_series(2,${TABLES_NUM}) AS id
    GROUP BY 1
    )
    SELECT CONCAT(stmt_start, stmt_end)
    FROM cte
    \gexec
EOM

##### Sub Queries

Tries to flatten queries

In [ ]:
# Sub Query
psql << EOM
SELECT *
FROM x, y,
    (SELECT * FROM a, b, c WHERE something) AS ss
WHERE somethingelse;
EOM

In [ ]:
# Flatten
psql << EOM
    SELECT * FROM x, y, a, b, c WHERE something AND somethingelse;
EOM

Aspects
- Pro: Qualificators can be propagated into the sub query
- Cons: More time planning because there are more options